In [ ]:
%pip install -U hue-entertainment-pykit

Experiment with the entertainment streaming library for Hue

# Enroll bridge

Code will enroll when no username / clientkey are available.

In [15]:
from ipaddress import ip_address
import requests
import urllib3
import json
import socket

# Hue Bridges have self-signed certs
urllib3.disable_warnings()


bridge_address = "philips-hue"
bridge_username = "vdjr8O4YGxVFALSPoyL-mMH09CLvQfWOsvuil38H"
bridge_clientkey = "1913F4CD82BF0834C8BC81B4074AB2BD"

while not bridge_username or not bridge_clientkey:
    # enrol per https://developers.meethue.com/develop/hue-entertainment/hue-entertainment-api/
    # get the username and client key from the Hue Bridge
    # Note: First request will always fail but puts bridge in a mode where the button can be pressed
    print(f"Enrolling at {bridge_address}.  Press button on bridge.")
    response: requests.Response = requests.post(
        "https://" + bridge_address + "/api", 
        data=json.dumps({
            "devicetype": f"funbiance#{socket.gethostname()}",
            "generateclientkey": True
        }),
        verify=False)
    print(f"Enrollment: {response.json()}")
    
    if response.status_code == 200:
        data = response.json()
        if "success" in data[0]:
            bridge_username = data[0]["success"]["username"]
            bridge_clientkey = data[0]["success"]["clientkey"]
        
else:
    print("Using existing username and client key")


Using existing username and client key


# Use REST API to gather info

In [24]:

bridge_params: dict[str, str] = {
    "username": bridge_username,
    "clientkey": bridge_clientkey
}

# hue_entertainment_pykit wants an IP address not a hostname
try:
    bridge_params["ip_address"] = socket.gethostbyname(bridge_address)
except:
    # Perhaps we already have an IP address?
    bridge_params["ip_address"] = bridge_address

# Define the URL and headers
headers: dict[str, str] = {
    "hue-application-key": bridge_username
}

# The "bridge" resource has some of the necessary information.
response: requests.Response = requests.get(f"https://{bridge_address}/clip/v2/resource/bridge", headers=headers, verify=False)

# Check if request was successful
if response.status_code == 200:
    # Parse JSON response
    data = response.json()
    print(data)
    
    bridge_params["identification"] = data["data"][0]["id"]
    bridge_params["rid"] = data["data"][0]["owner"]["rid"]
    
else:
    print(f"Request failed with status code: {response.status_code}")
    print(response.text)
    
response: requests.Response = requests.get(f"https://{bridge_address}/api/config", headers=headers, verify=False)

# Check if request was successful
if response.status_code == 200:
    # Parse JSON response
    data = response.json()
    print(data)
    
    bridge_params["swversion"] = int(data["swversion"])
    bridge_params["name"] = data["name"]
    
else:
    print(f"Request failed with status code: {response.status_code}")
    print(response.text)
    
    
response: requests.Response = requests.get(f"https://{bridge_address}/auth/v1", headers=headers, verify=False)

# Check if request was successful
if response.status_code == 200:
    print(response.headers)
    bridge_params["hue_app_id"] = response.headers["hue-application-id"]
    
else:
    print(f"Request failed with status code: {response.status_code}")
    print(response.text)
    
print("Bridge params:", json.dumps(bridge_params, indent=4))

{'errors': [], 'data': [{'id': '16d639ef-f8f6-4d56-b1dc-651b1c31af4e', 'owner': {'rid': '9ac3ae4b-c411-4e98-8a87-d2332926b918', 'rtype': 'device'}, 'bridge_id': 'ecb5fafffe172def', 'time_zone': {'time_zone': 'America/Vancouver'}, 'type': 'bridge'}]}
{'name': 'Hue Bridge', 'datastoreversion': '172', 'swversion': '1967054020', 'apiversion': '1.67.0', 'mac': 'ec:b5:fa:17:2d:ef', 'bridgeid': 'ECB5FAFFFE172DEF', 'factorynew': False, 'replacesbridgeid': None, 'modelid': 'BSB002', 'starterkitid': ''}
Bridge params: {
    "username": "vdjr8O4YGxVFALSPoyL-mMH09CLvQfWOsvuil38H",
    "clientkey": "1913F4CD82BF0834C8BC81B4074AB2BD",
    "ip_address": "172.16.0.4",
    "identification": "16d639ef-f8f6-4d56-b1dc-651b1c31af4e",
    "rid": "9ac3ae4b-c411-4e98-8a87-d2332926b918",
    "swversion": 1967054020,
    "name": "Hue Bridge",
    "hue_app_id": "235603fa-56ea-45c1-9def-b1acb2f04145"
}


In [34]:
import time
from hue_entertainment_pykit import create_bridge, Entertainment, Streaming, Bridge

# Set up the Bridge instance with the all needed configuration
# bridge = create_bridge(
    # identification="4abb74df-5b6b-410e-819b-bf4448355dff",
    # rid="d476df48-83ad-4430-a104-53c30b46b4d0",
    # ip_address="192.168.1.100",
    # swversion=1962097030,
    # username="8nuTIcK2nOf5oi88-5zPvV1YCt0wTHZIGG8MwXpu",
    # hue_app_id="94530efc-933a-4f7c-97e5-ccf1a9fc79af",
    # clientkey="B42753E1E1605A1AB90E1B6A0ECF9C51",
    # name="1st Bridge"
# )
bridge: Bridge = create_bridge(**bridge_params)

# Set up the Entertainment API service
entertainment_service = Entertainment(bridge)

# Fetch all Entertainment Configurations on the Hue bridge
entertainment_configs = entertainment_service.get_entertainment_configs()
for id in entertainment_configs:
    print(f"Entertainment configurations: {id} = {entertainment_configs[id].name}")
    for channel in entertainment_configs[id].channels:
        print(f"\tchannel: {channel.channel_id} position={channel.position}")

# Add some Entertainment Area selection logic
# For the purposes of example I'm going to do manual selection
entertainment_config = list(entertainment_configs.values())[1]

# Set up the Streaming service
streaming = Streaming(
    bridge, entertainment_config, entertainment_service.get_ent_conf_repo()
)

# Start streaming messages to the bridge
streaming.start_stream()

# Set the color space to xyb or rgb
streaming.set_color_space("xyb")

# Set input commands for the lights
# First three values in the tuple are placeholders for the color RGB8(int) or (in this case) XYB(float) and the last integer is light ID inside the Entertainment API
streaming.set_input((0.0, 0.63435, 0.3, 0))  # Light command for the first light
streaming.set_input((0.63435, 0.0, 0.3, 1))  # Light command for the second light
# ... Add more inputs as needed for additional lights and logic

# For the purpose of example sleep is used for all inputs to process before stop_stream is called
# Inputs are set inside Event queue meaning they're on another thread so user can interact with application continuously
time.sleep(0.1)

# Stop the streaming session
streaming.stop_stream()

Entertainment configurations: 4fb05ad4-adb2-4a2e-b47f-41b1ca19bcce = TV Area
	channel: 0 position=Position(x=0.93997, y=1.0, z=-0.00018)
	channel: 1 position=Position(x=-1.0, y=1.0, z=-0.42379)
	channel: 2 position=Position(x=-0.05998, y=1.0, z=0.56998)
	channel: 3 position=Position(x=-0.54, y=1.0, z=-0.62708)
	channel: 4 position=Position(x=0.35705, y=1.0, z=-0.56941)
Entertainment configurations: 9c611527-6163-4470-a385-2968ae9970bc = Office Area
	channel: 0 position=Position(x=-0.05172, y=1.0, z=-1.0)
	channel: 1 position=Position(x=-1.0, y=1.0, z=-0.84327)
